In [ ]:
from pygame import mixer
import time

# Initialize mixer
mixer.init()

# Get song name from user
song = input("What song do you want to play? > ").strip().lower()

try:
    mixer.music.load(song + ".mp3")  # Load song
    mixer.music.set_volume(0.5)  # Default volume set to 50%
    mixer.music.play()  # Play song
    start_time = time.time()  # Track song start time

    print("\nControls:")
    print("Press 'p' to pause")
    print("Press 'r' to resume")
    print("Press 'v' to set volume manually")
    print("Press '+' to increase volume")
    print("Press '-' to decrease volume")
    print("Press '>' to skip forward 10 seconds")
    print("Press '<' to rewind 10 seconds")
    print("Press 'e' to exit")

    while True:
        ch = input("['p','r','v','+','-','>','<','e']>>> ").strip().lower()

        if ch == "p":
            mixer.music.pause()
            print("Music paused")
        elif ch == "r":
            mixer.music.unpause()
            print("Music resumed")
        elif ch == "v":
            v = float(input("Enter volume (0 to 10): ")) / 10
            mixer.music.set_volume(v)
            print(f"Volume set to {int(v * 10)}")
        elif ch == "+":
            current_vol = mixer.music.get_volume()
            new_vol = min(current_vol + 0.1, 1.0)  # Ensure max is 1.0
            mixer.music.set_volume(new_vol)
            print(f"Volume increased to {int(new_vol * 10)}")
        elif ch == "-":
            current_vol = mixer.music.get_volume()
            new_vol = max(current_vol - 0.1, 0.0)  # Ensure min is 0.0
            mixer.music.set_volume(new_vol)
            print(f"Volume decreased to {int(new_vol * 10)}")
        elif ch == ">":
            current_pos = mixer.music.get_pos() / 1000  # Get current time in seconds
            new_pos = current_pos + 10
            mixer.music.play(start=int(new_pos))
            print(f"Skipped forward to {int(new_pos)} seconds")
        elif ch == "<":
            current_pos = mixer.music.get_pos() / 1000  # Get current time in seconds
            new_pos = max(0, current_pos - 10)  # Ensure it doesn't go negative
            mixer.music.play(start=int(new_pos))
            print(f"Rewound to {int(new_pos)} seconds")
        elif ch == "e":
            mixer.music.stop()
            print("Exiting...")
            break
        else:
            print("Invalid input, try again!")

except Exception as e:
    print(f"Error: {e}. Make sure the MP3 file exists!")


In [ ]:
# Import necessary modules
import requests
from tkinter import filedialog, Tk, Menu, Listbox, Button, Frame, PhotoImage, END
import pygame
import os

# Initialize the Tkinter window
app = Tk()
app.title('Music Player')
app.geometry("500x300")

# Change the application icon
app_icon = PhotoImage(file='background.png') 
app.iconphoto(False, app_icon)

# Initialize Pygame's mixer module for playing audio
pygame.mixer.init()

# Define an event for when a song ends
SONG_END_EVENT = pygame.USEREVENT + 1
pygame.mixer.music.set_endevent(SONG_END_EVENT)

# Create a menu bar
menu_bar = Menu(app)
app.config(menu=menu_bar)

# Define global variables
playlist = []  # List to store names of songs
current_song = ""  # Store the currently playing song
is_paused = False  # Flag to indicate if music is paused

#Necessary functions___________________________________________________________________
#function to load music from a directory
def load_music():
    global current_song
    app.directory = filedialogue.askdirectory()

    # Clear the current list of songs
    playlist.clear()
    song_listbox.delete(0, END)

    for file in os.listdir(app.directory):
        name, ext = os.path.splitext(file)
        if ext == '.mp3':
            playlist.append(file)

    for song in playlist:
        song_listbox.insert("end", song)

    #By default, starts playing the first song
    if playlist:
        song_listbox.selection_set(0)
        current_song = playlist[song_listbox.curselection()[0]]

def play_music(event=None):
    global current_song, is_paused

    if not playlist:
        return

    #Gets me the selected song from my pop up window
    current_selection = song_listbox.curselection()
    if current_selection:
        current_song = playlist[current_selection[0]]

    if not is_paused:
        mixer.music.load(os.path.join(app.directory, current_song))
        mixer.music.play()
    else:
        mixer.music.unpause()
        is_paused = False

#Tkinter_______________________________________________________________________________

# Create a menu for adding songs
add_songs_menu = Menu(menu_bar, tearoff=False)
add_songs_menu.add_command(label='Select Folder', command=load_music)
menu_bar.add_cascade(label='Add Songs', menu=add_songs_menu)

# Create a listbox to display songs
song_listbox = Listbox(app, bg="green", fg="white", width=100, height=13)
song_listbox.pack()

# Bind a selection event to the listbox
song_listbox.bind("<<ListboxSelect>>", play_music)

# Load images for control buttons
play_button_image = PhotoImage(file='play.png')
pause_button_image = PhotoImage(file='pause.png')
next_button_image = PhotoImage(file='next.png')
previous_button_image = PhotoImage(file='previous.png')

# Create control buttons
control_frame = Frame(app)
control_frame.pack()

play_button = Button(control_frame, image=play_button_image, borderwidth=0, command=play_music)
pause_button = Button(control_frame, image=pause_button_image, borderwidth=0, command=pause_music)
next_button = Button(control_frame, image=next_button_image, borderwidth=0, command=next_song)
previous_button = Button(control_frame, image=previous_button_image, borderwidth=0, command=previous_song)

# Arrange control buttons
previous_button.grid(row=0, column=0, padx=5)
play_button.grid(row=0, column=1, padx=5)
pause_button.grid(row=0, column=2, padx=5)
next_button.grid(row=0, column=3, padx=5)

# Start checking for the end of song event
app.after(100, check_music_end)

# Start Tkinter event loop
app.mainloop()
